In [ ]:
from datasets import load_dataset
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import os
import re
import google.generativeai as genai
import time


# -------- Setup Gemini --------
GEMINI_API_KEY = "YOUR_API_KEY"
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash")  # nhanh và rẻ


ds = load_dataset("tmnam20/ViMedAQA", "drug")



df_train = ds['train'].to_pandas()
df_test = ds['test'].to_pandas()
df_validation = ds['validation'].to_pandas()

In [ ]:
def get_pattern_from_gemini(title_sample: str, retries=3) -> str:
    """
    Gọi Gemini để tạo ra SQL LIKE pattern tổng quát cho tiêu đề.
    Có retry và fallback nếu lỗi.
    """
    prompt = f"""
    Bạn được cung cấp một tiêu đề dữ liệu: "{title_sample}".

    Hãy tạo ra một mẫu SQL LIKE pattern CHUNG CHUNG để khớp với tất cả tiêu đề có cùng cấu trúc,
    tuân thủ các quy tắc sau:
    - Chỉ dùng dấu % ở ĐẦU hoặc CUỐI chuỗi, không dùng dạng %abc%.
    - Không đưa các từ cụ thể (như tên thuốc, tên người, số hiệu...) vào pattern.
    - Pattern chỉ mô tả dạng tiêu đề (ví dụ: "Tác dụng phụ%", "Hướng dẫn sử dụng%", ...).
    - Trả về duy nhất chuỗi pattern, không thêm giải thích hoặc văn bản thừa.

    Chỉ trả về pattern duy nhất, không kèm giải thích.
    """

    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            pattern = response.text.strip()
            # Làm sạch output
            pattern = pattern.replace("`", "").replace('"', "").replace("'", "")
            pattern = re.sub(r"\s+", " ", pattern).strip()
            return pattern
        except Exception as e:
            print(f"[WARN] Gemini call failed (attempt {attempt+1}/{retries}): {e}")
            if attempt < retries - 1:
                print("Chờ 10 giây và thử lại...")
                time.sleep(10)
            else:
                print("[FALLBACK] Không gọi được Gemini, dùng prefix đơn giản.")
                return simple_pattern(title_sample)

def simple_pattern(title: str) -> str:
    """Fallback: Trích cụm đầu tiên trước dấu ':' hoặc '-'."""
    prefix = re.split(r"[:\-]", title)[0].strip()
    return prefix + "%"

def like_filter(df: pd.DataFrame, column: str, pattern: str) -> pd.DataFrame:
    """Lọc dataframe dựa trên SQL LIKE pattern, chuyển thành regex để match."""
    regex = re.escape(pattern).replace("%", ".*")
    return df[df[column].str.match(regex, na=False)]

# -------- Load Previous Progress --------
if os.path.exists("pattern_counts_partial.csv"):
    results = pd.read_csv("pattern_counts_partial.csv").to_dict(orient="records")
    print(f"[RESUME] Đọc {len(results)} pattern đã xử lý từ file tạm.")
else:
    results = []

# Xác định những title đã xử lý để bỏ qua
processed_patterns = [r["pattern"] for r in results]

# -------- Main Loop --------
remaining = df_train.copy()

# Loại bỏ các dòng đã match với pattern trước đó
for p in processed_patterns:
    matched = like_filter(remaining, "title", p)
    remaining = remaining.drop(matched.index)

iteration = len(results)

try:
    while not remaining.empty:
        iteration += 1
        print(f"Iteration {iteration}, remaining rows: {len(remaining)}")

        first_title = remaining.iloc[0]["title"]
        pattern = get_pattern_from_gemini(first_title)
        print(f"Pattern Gemini tìm thấy: {pattern}")

        matched = like_filter(remaining, "title", pattern)
        count = len(matched)
        results.append({"pattern": pattern, "count": count})

        remaining = remaining.drop(matched.index)

        # Lưu tạm sau mỗi vòng lặp
        pd.DataFrame(results).to_csv("drug_train_pattern_counts_partial.csv", index=False)

        # Delay tránh quota
        time.sleep(6)

except KeyboardInterrupt:
    print("[STOP] Người dùng dừng thủ công. Lưu kết quả tạm thời...")
except Exception as e:
    print(f"[ERROR] Lỗi bất ngờ: {e}. Lưu kết quả tạm thời...")

# -------- Save Final Results --------
results_df = pd.DataFrame(results)
results_df.to_csv("drug_train_pattern_counts.csv", index=False)
print("Đã lưu kết quả ra drug_train_pattern_counts.csv")